In [1]:
# Importação das bibliotecas necessárias
import pandas as pd

from bs4 import BeautifulSoup
import requests

In [2]:
fonte = 'mapbiomas'
dataset_name = 'cobertura_solo'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [3]:
# Requisição para a URL
response = requests.get('https://brasil.mapbiomas.org/estatisticas/')

# Criação do objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Listagem de todos os links na página
links = soup.find_all('a')

# Filtro para apenas os links de download
download_links = [link.get('href') for link in links if 'download' in link.get('href') and 'COBERTURA' in link.text and 'MUNICÍPIO' in link.text]

# Seleção do link de download (caso o filtro pegue outros links que atendam o critério) com aquisição apenas do nome do arquivo
cobertura_xlsx = download_links[0].split("/")[-1]

# Salvamento do arquivo em disco
with open(f'{raiz}/bronze_{cobertura_xlsx}', 'wb') as arquivo:
    arquivo.write(requests.get(download_links[0]).content)

### Prata

In [4]:
# Obtenção de lista com anos de interesse que constam como colunas da tabela
anos = (pd.read_excel(f'{raiz}/bronze_{cobertura_xlsx}',
                      sheet_name = 1,
                      usecols=lambda x: isinstance(x, int))
        .columns
        .drop(range(1985, 1990))
        .tolist()
        )

In [6]:
df = (
    pd.read_excel(f'{raiz}/bronze_{cobertura_xlsx}',
                      sheet_name = 1)
    .drop(range(1985, 1990), axis = 'columns')
    .rename(columns=lambda col: 'class' if col == 'class_id' else col)
    .melt(
        id_vars=['geocode', 'class'],
        value_vars=anos,
        var_name='data',
        value_name='area'
        )
    .dropna(subset='area')
    .query('area != 0.0')
    .assign(data = lambda df: pd.to_datetime(df.data, format='%Y'), # Conversão de número inteiro para formato de data seguindo a ISO 8601
                                area = lambda x: x.area/1000000) 
    .convert_dtypes()
    .rename(columns=
            {
                'geocode':'geocodigo',
                'class':'classe_solo'
             })
    )
(
    df.set_index(['geocodigo', 'data', 'classe_solo'])
    .to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_municipio.parquet')
)

df_uf = (df
         .assign(geocodigo_uf = lambda df: (df['geocodigo']// 100000))
         .groupby(['geocodigo_uf', 'data', 'classe_solo'])
         .agg({'area':sum})
         .rename_axis(index={'geocodigo_uf': 'geocodigo'})
         .convert_dtypes()
         )
df_uf.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_uf.parquet')